# Введение в Ansible

Этот ноутбук — рабочий материал по настройке стека приложений с помощью Ansible


## Цели

1. Научиться доставлять код в созданную инфраструктуру.


In [ ]:
%%capture
%%sh
pip install ansible -qqq
sudo apt update && sudo apt install -y tree
mkdir ansible_quickstart

In [ ]:
%%writefile ansible_hack.reg
Windows Registry Editor Version 5.00

[HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\FileSystem]
"LongPathsEnabled"=dword:00000001

Writing ansible_hack.reg


### Параметры подключения

Параметры подключения лучше хранить в файле /etc/ansible/hosts

In [ ]:
# @markdown ввод параметров подключения

ansible_ssh_host = "127.0.0.1"   # @param {'type':'string'}
ansible_user = "root"       # @param {'type':'string'}
ansible_password = "root"   # @param {'type':'string'}

In [ ]:
%cd ansible_quickstart

/content/ansible_quickstart


## Конвейер развёртывания приложения на удалённой машине

### Создаём файл инвентаря

Файл инвентаря создаётся на узле управления (контроллере)

In [ ]:
import os

inventory_content = f"""myhosts:
  hosts:
    my_host_01:
      ansible_host: {ansible_ssh_host}
      ansible_user: {ansible_user}
      ansible_password: "{ansible_password}"
"""

with open("inventory.yaml", "w") as f:
    f.write(inventory_content)

In [ ]:
!ansible-inventory -i inventory.yaml --graph myhosts

@myhosts:
  |--my_host_01


In [ ]:
%%writefile ansible.cfg
[defaults]
host_key_checking = False

Writing ansible.cfg


In [ ]:
!ansible -m ping localhost #-vvvv

[WARNING]: No inventory was parsed, only implicit localhost is available
localhost | SUCCESS => {
    "changed": false,
    "ping": "pong"
}


In [ ]:
!cat inventory.yaml
!ansible-inventory -i inventory.yaml --list
!ansible-inventory -i inventory.yaml --list hosts

myhosts:
  hosts:
    my_host_01:
      ansible_host: 147.45.215.174
      ansible_user: root
      ansible_password: "vbRnKW3?QT96M^"
{
    "_meta": {
        "hostvars": {
            "my_host_01": {
                "ansible_host": "147.45.215.174",
                "ansible_password": "vbRnKW3?QT96M^",
                "ansible_user": "root"
            }
        },
        "profile": "inventory_legacy"
    },
    "all": {
        "children": [
            "ungrouped",
            "myhosts"
        ]
    },
    "myhosts": {
        "hosts": [
            "my_host_01"
        ]
    }
}
{
    "_meta": {
        "hostvars": {
            "my_host_01": {
                "ansible_host": "147.45.215.174",
                "ansible_password": "vbRnKW3?QT96M^",
                "ansible_user": "root"
            }
        },
        "profile": "inventory_legacy"
    },
    "all": {
        "children": [
            "ungrouped",
            "myhosts"
        ]
    },
    "myhosts": {
        "ho

### Создаём плейбук

In [ ]:
%%writefile playbook.yaml
---
- name: Запуск сервера одной командой
  hosts: localhost
  become: yes
  gather_facts: yes

  vars:
    app_dir: .
    app_file: "{{ app_dir }}/main.py"
    service_name: fastapi-app
    server_port: 8000

  tasks:
    - name: Проверка, что директория существует
      file:
        path: "{{ app_dir }}"
        state: directory
        mode: '0755'

    - name: Создание приложения FastAPI (main.py)
      copy:
        content: |
          from fastapi import FastAPI
          app = FastAPI(title="Minimal FastAPI Server", version="1.0")

          @app.get("/health")
          def health_check():
              return {"status": "OK"}
        dest: "{{ app_file }}"
        mode: '0644'

    - name: Установка pip
      apt:
        name:
          - python3
          - python3-pip
          - python3-venv
        state: present
        update_cache: yes

    - name: Установка FastAPI и Uvicorn с помощью pip
      pip:
        name:
          - fastapi
          - uvicorn
        state: present
        executable: pip3

    - name: Запуск сервера FastAPI в фоновом режиме
      command: uvicorn main:app --host 0.0.0.0 --port "{{ server_port }}" --reload
      args:
        chdir: "{{ app_dir }}"
      async: 10
      poll: 0
      register: fastapi_process

    - name: Ожидание запуска сервера FastAPI (до 35 секунд)
      wait_for:
        host: 127.0.0.1
        port: "{{ server_port }}"
        timeout: 35

    - name: Проверка работоспособность с помощью curl
      uri:
        url: "http://127.0.0.1:{{ server_port }}/health"
        method: GET
        return_content: yes
      register: health_response

    - name: Печать результата проверки работоспособности
      debug:
        msg: "результат проверки работоспособности: {{ health_response.content }}"

    - name: Печать информацию о порте сервера
      debug:
        msg: "Открыт порт сервера http://127.0.0.1:{{ server_port }}/health"

Writing playbook.yaml


### Отправляем плейбук на управляемый узел

In [ ]:
#!cat playbook.yaml
!ansible-playbook playbook.yaml -i "localhost,"  --connection=local -e "server_port=8000"


PLAY [Запуск сервера одной командой] *******************************************

TASK [Gathering Facts] *********************************************************
[WARNING]: Host 'localhost' is using the discovered Python interpreter at '/usr/bin/python3.12', but future installation of another Python interpreter could cause a different interpreter to be discovered. See https://docs.ansible.com/ansible-core/2.19/reference_appendices/interpreter_discovery.html for more information.
ok: [localhost]

TASK [Проверка, что директория существует] *************************************
ok: [localhost]

TASK [Создание приложения FastAPI (main.py)] ***********************************
changed: [localhost]

TASK [Установка pip] ***********************************************************
changed: [localhost]

TASK [Установка FastAPI и Uvicorn с помощью pip] *******************************
ok: [localhost]

TASK [Запуск сервера FastAPI в фоновом режиме] *********************************
changed: [loca

In [ ]:
!ansible myhosts -m ping -i inventory.yaml --ssh-extra-args="-o StrictHostKeyChecking=no"

[WARNING]: Host 'my_host_01' is using the discovered Python interpreter at '/usr/bin/python3.10', but future installation of another Python interpreter could cause a different interpreter to be discovered. See https://docs.ansible.com/ansible-core/2.19/reference_appendices/interpreter_discovery.html for more information.
my_host_01 | SUCCESS => {
    "ansible_facts": {
        "discovered_interpreter_python": "/usr/bin/python3.10"
    },
    "changed": false,
    "ping": "pong"
}


In [ ]:
!ansible-playbook /content/ansible_quickstart/playbook.yaml -i inventory.yaml  -e "server_port=8010"


PLAY [Запуск сервера одной командой] *******************************************

TASK [Gathering Facts] *********************************************************
ok: [localhost]

TASK [Проверка, что директория существует] *************************************
ok: [localhost]

TASK [Создание приложения FastAPI (main.py)] ***********************************
ok: [localhost]

TASK [Установка pip] ***********************************************************
ok: [localhost]

TASK [Установка FastAPI и Uvicorn с помощью pip] *******************************
ok: [localhost]

TASK [Запуск сервера FastAPI в фоновом режиме] *********************************
changed: [localhost]

TASK [Ожидание запуска сервера FastAPI (до 35 секунд)] *************************
ok: [localhost]

TASK [Проверка работоспособность с помощью curl] *******************************
ok: [localhost]

TASK [Печать результата проверки работоспособности] ****************************
ok: [localhost] => {
    "msg": "результат пров

### Шаблон настройки роли

In [ ]:
!ansible-galaxy init nginx --force
!tree /content/ansible_quickstart/nginx

- Role nginx was created successfully
/content/ansible_quickstart/nginx
├── defaults
│   └── main.yml
├── files
├── handlers
│   └── main.yml
├── meta
│   └── main.yml
├── README.md
├── tasks
│   └── main.yml
├── templates
├── tests
│   ├── inventory
│   └── test.yml
└── vars
    └── main.yml

8 directories, 8 files


## Защита паролей с помощью ansible-vault



### Создаём нешифрованный YAML-файл с секретом

In [ ]:
%%writefile temp_secret.yml
ansible_password: mysupersecretpassword123
db_host: 192.168.1.10
db_port: 5432

Writing temp_secret.yml


In [ ]:
!ansible-vault encrypt --vault-id vault_password.txt@prompt temp_secret.yml
!mv temp_secret.yml secret.yml

New vault password (vault_password.txt): 
Confirm new vault password (vault_password.txt): 
Encryption successful


### Шифрованный файл сохраняется как secret.yml — его можно в таком виде спокойно коммитить в Git (расшифровать без пароля невозможно)

In [ ]:
!cat secret.yml

$ANSIBLE_VAULT;1.2;AES256;vault_password.txt
33646332656638373539646630323538656133383062356238356433393764393162303232613763
6235373333626264366439643332326438613936393134380a663566633036353265366137383564
36636161363063363831636632613232396339353163663963373636333165326134633033616166
3765363934353832330a633737383665613933313738663065653839353636303532626231383335
38656534613230653937613432343635383132653630626237363535373434366461653666316639
32366262666165303832633939363933656233366565303735363765316238373731313664643431
33376532333065336261376334306235363965333965346664383037613935346630386365343361
33393432656634633730


### Расшифровываем с паролем

In [ ]:
!ansible-vault view --vault-id vault_password.txt@prompt secret.yml

Vault password (vault_password.txt): 
ansible_password: mysupersecretpassword123
db_host: 192.168.1.10
db_port: 5432


## Выводы

Сколько можно одновременно запустить серверов в автоматическом режиме ?

**Сколько угодно !** (до исчерпания ресурсов физического сервера)